In [1]:
import tensorflow as tf
import numpy as np

### Define the AND gate

In [2]:
# Logical AND operator
X = [[0., 0.], [0., 1.], [1., 0.], [1., 1.]]
Y = [[0.], [0.], [0.], [1.]]

### Start Session

In [3]:
sess = tf.Session()

### Check variable assignments

In [4]:
x = tf.placeholder(tf.float32, shape=(None,2), name='input')
sess.run(x, {x: X})

array([[ 0.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  1.]], dtype=float32)

In [5]:
sess.run(x, feed_dict={x: X})

array([[ 0.,  0.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  1.]], dtype=float32)

In [6]:
W = tf.Variable([[2.],[0.]], tf.float32, name='W')
W

In [7]:
c = tf.matmul(x,W)
c

<tf.Tensor 'MatMul:0' shape=(?, 1) dtype=float32>

In [8]:
init = tf.global_variables_initializer()
sess.run(init) # reset values 
sess.run([c], {x: X})

[array([[ 0.],
        [ 0.],
        [ 2.],
        [ 2.]], dtype=float32)]

In [9]:
with tf.name_scope('losses'):
    A = [1.,-1.,1.,0.]
    loss1 = tf.nn.l2_loss(A)               #  = MSE*num_samples / 2
    loss2 = tf.reduce_sum(tf.square(A))    #  = MSE*num_samples
    loss3 = tf.reduce_mean(tf.square(A))   #  = MSE
sess.run([loss1, loss2, loss3])

[1.5, 3.0, 0.75]

#### Write to tensorboard

In [10]:
writer = tf.summary.FileWriter('/tmp/logic_gate_and/3')
writer.add_graph(sess.graph)

In anaconda prompt,

\>> activate py35

\>> tensorboard --logdir /tmp/logic_gate_and/3

Enter the url (e.g.  http://192.168.0.6:6006 ) in the browser to open the tensorboard.

### AND gate with MSE loss

In [11]:
with tf.name_scope('logit_model'):
    tf.assign(W, [[.1],[-.1]])  # reassign W
    b = tf.Variable([.3], tf.float32, name='bias')
    logits = tf.matmul(x, W)+b

In [12]:
y = tf.placeholder(tf.float32, name='label')

In [13]:
with tf.name_scope('loss_mse'):
    loss = tf.reduce_mean(tf.square(logits - y)) # MSE

In [14]:
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(0.1)
    train = optimizer.minimize(loss)

In [15]:
# training loop
init = tf.global_variables_initializer()
sess.run(init) # reset values to wrong
for i in range(100):
    sess.run(train, {x:X, y:Y})  # implicit feed_dict argument
# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:X, y:Y})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))
sess.run([logits], {x:X, y:Y})

W: [[ 0.51012611]
 [ 0.49828494]] b: [-0.25498834] loss: 0.062527


[array([[-0.25498834],
        [ 0.24329659],
        [ 0.25513777],
        [ 0.75342274]], dtype=float32)]

In [16]:
writer = tf.summary.FileWriter('/tmp/logic_gate_and/4')
writer.add_graph(sess.graph)